# Getting Started

You can create vectors using `json_array_to_vector` and then you can calculate distance between to vectors using `json_array_to_vector`:

In [3]:
declare @v1 vector(3) = cast('[1,3,-5]' as vector(3));
declare @v2 vector(3) = cast('[4,-2,-1]' as vector(3));

select 
    vector_distance('euclidean', @v1, @v2) AS euclidean_distance,
    vector_distance('cosine', @v1, @v2) AS cosine_distance,
    vector_distance('dot', @v1, @v2) AS dotproduct_distance;

(1 row affected)

Total execution time: 00:00:00.084

euclidean_distance,cosine_distance,dotproduct_distance
7.0710678118654755,0.8893433329655024,-3


## Store vectors in tables

Vectors can be stored in a table. The function `isvector` can be used to make sure that the stored binary is actually a vector

In [7]:
drop table if exists #vectors
create table #vectors
(
    id int not null identity primary key,
    vector vector(3) not null
);

insert into #vectors (vector)
select 
    cast(a as vector(3))
from
    ( values 
        ('[4, -2, -1]'),
        ('[1, 3, -5]'),
        ('[7, -8, -10]'),
        ('[4.0, 0.2, -1.1]'),
        ('[0, 0, 0]'),
        ('[10, 10, 10]'),
        ('[-0.1, -0.2, -0.3]')
    ) V(a)
;

select id, cast(vector as varchar(100)) as v from #vectors

(7 rows affected)

(7 rows affected)

Total execution time: 00:00:00.066

id,v
1,"[4.0000000e+000,-2.0000000e+000,-1.0000000e+000]"
2,"[1.0000000e+000,3.0000000e+000,-5.0000000e+000]"
3,"[7.0000000e+000,-8.0000000e+000,-1.0000000e+001]"
4,"[4.0000000e+000,2.0000000e-001,-1.1000000e+000]"
5,"[0.0000000e+000,0.0000000e+000,0.0000000e+000]"
6,"[1.0000000e+001,1.0000000e+001,1.0000000e+001]"
7,"[-1.0000000e-001,-2.0000000e-001,-3.0000001e-001]"


# Query vectors in tables

Vector distance can be used with any valid expression that evaluate to a vector. For example it can be used to calcualte distance between vectors in a table

In [8]:
declare @v vector(3) = cast('[6.4,-2.5,1.3]' as vector(3));
 
select 
    v.id,    
    vector_distance('cosine', @v, v.vector) as distance
from
    #vectors v
where
    vector_distance('cosine', @v, v.vector) < 0.5
order by
    distance


(3 rows affected)

Total execution time: 00:00:00.058

id,distance
1,0.08566878877110973
4,0.18501562892002543
3,0.4924421685441406


# Vector Dimensions

In order to calculate distance, vectors must have the same number of dimensions. The following query will fail as the query vector `@v` has 6 dimensions while the stored vectors have only 3.

In [9]:
declare @v vector(6) = cast('[1, 2, 3, 4, 5, 6]' as vector(6));
 
select 
    v.id,    
    vector_distance('cosine', @v, v.vector) as distance
from
    #vectors v
where
    vector_distance('cosine', @v, v.vector) < 0.5
order by
    distance

Total execution time: 00:00:00.050

id,distance


: Msg 42204, Level 16, State 3, Line 3
VECTOR dimensions 6 and 3, do not match.